In [201]:
import pandas as pd
import numpy as np
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler

import nltk
# from nltk import word_tokenize - нужно nltk.download('punkt')

from nltk import wordpunct_tokenize, wordnet
from nltk.stem import wordnet as WordNetLem
from nltk.stem import SnowballStemmer, StemmerI

import gensim
from gensim.corpora import Dictionary
from gensim.models import doc2vec

In [179]:
stop_words = \
    ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 
     'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 
     'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 
     'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 
     'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 
     'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 
     'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 
     'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 
     'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 
     'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 
     'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', 'том', 'нельзя', 'такой', 
     'им', 'более', 'всегда', 'конечно', 'всю', 'между']
stop_words.extend(['очень', 'ооочень', 'это', 'данное'])

# Загрузка данных

In [163]:
df = pd.read_csv('data/coffee.csv')
df['rating'] = df['rating'].astype('float16')

In [164]:
df_tmp = df[df['rating'] < 3].iloc[:500]
df_tmp.shape

(500, 5)

In [203]:
df_tmp['text'].iloc[:3]

3     Самый большой плюс это месторасположение, набе...
6     1. Доставка очень долгая, на рекламной брошюре...
15    Заказали фо-бо и том-ям\nКороче больше не прид...
Name: text, dtype: object

In [206]:
# распределение отзывов по рейтингу - не сбаллансированные классы
df['rating'].value_counts()

5.0    88530
4.0    10811
1.0     8200
3.0     6335
2.0     3893
0.0       49
Name: rating, dtype: int64

In [205]:
# выделение предложений и оборотов
pattern = r".+?[,.?!()]"
result = []
for text_x in df_tmp['text']:
    result_tmp = re.findall(pattern, text_x)
    result.append(result_tmp)
result

[['Самый большой плюс это месторасположение,',
  ' набережная ,',
  ' шикарный вид на море!',
  ' Красиво,',
  ' уютно,',
  ' вот собственно плюсы закончились .',
  '. огорчает отношение к посетителям,',
  ' официанты неприветливые,',
  ' не здравствуйте вам,',
  ' не до свидания .',
  ' Лица недовольные,',
  ' неприятные,',
  ' больше не хочется смотреть на такие!',
  ' Кухня тоже оставляет желать лучшего,',
  ' в люле кебаб кости попадаются,',
  ' шашлык из говядины сухой и невкусный.',
  ' Мы на отдыхе ,',
  ' на позитиве ,',
  ' денег не жалеем,',
  ' но хочется приходить туда где нам рады!'],
 ['1.',
  ' Доставка очень долгая,',
  ' на рекламной брошюре написано от  500 р доставка бесплатно,',
  ' а по факту заплатили 100 р за доставку.',
  ' Чек был на 577р\\n2.',
  ' Долго плевались от нагара который был на сковородке (',
  'подноса)',
  ' с мясом,',
  ' хруст на зубах не приятен.',
  ' \\nИспорченное мнение от вашего заведения.'],
 ['Заказали фо-бо и том-ям\\nКороче больше не п

# Предобработка данных 
- токенизация gensim, потому что там без пунктуации получается
- стемминг/лемматизация
- удаление стоп слов
- создание би/три-грамм

In [170]:
# лемматизация
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [171]:
# есть проблема - nчто можно улучшить - маловато места посадки. n английская остается.
tokens = list(sent_to_words(df_tmp['text']))
tokens[0][:32]

['самыи',
 'большои',
 'плюс',
 'это',
 'набережная',
 'шикарныи',
 'вид',
 'на',
 'море',
 'красиво',
 'уютно',
 'вот',
 'собственно',
 'плюсы',
 'закончились',
 'огорчает',
 'отношение',
 'посетителям',
 'официанты',
 'неприветливые',
 'не',
 'здравствуите',
 'вам',
 'не',
 'до',
 'свидания',
 'лица',
 'недовольные',
 'неприятные',
 'больше',
 'не',
 'хочется']

In [180]:
# стемминг и удаление стоп слов
stemmer = SnowballStemmer('russian')

stem_nltk = []
for sentence in tokens:
    stem_nltk.append(list(stemmer.stem(word_x) for word_x in sentence if word_x not in stop_words))
stem_nltk[0], len(stem_nltk[0])

(['самы',
  'большо',
  'плюс',
  'набережн',
  'шикарны',
  'вид',
  'мор',
  'красив',
  'уютн',
  'собствен',
  'плюс',
  'законч',
  'огорча',
  'отношен',
  'посетител',
  'официант',
  'неприветлив',
  'здравству',
  'свидан',
  'лиц',
  'недовольн',
  'неприятн',
  'хочет',
  'смотрет',
  'так',
  'кухн',
  'оставля',
  'жела',
  'лучш',
  'люл',
  'кебаб',
  'кост',
  'попада',
  'шашлык',
  'говядин',
  'сухо',
  'невкусны',
  'отдых',
  'позитив',
  'денег',
  'жале',
  'хочет',
  'приход',
  'туд',
  'нам',
  'рад'],
 46)

# Кодирование текста
- CountVectorizer
- CountVectorizer(binary=True)
- Tfidf
- Doc2Vec

In [181]:
# объединим текст в отзывы
tok_stem_text = []
for word_x in range(len(stem_nltk)):
    tok_stem_text.append(" ".join(stem_nltk[word_x]))
tok_stem_text[0]

'самы большо плюс набережн шикарны вид мор красив уютн собствен плюс законч огорча отношен посетител официант неприветлив здравству свидан лиц недовольн неприятн хочет смотрет так кухн оставля жела лучш люл кебаб кост попада шашлык говядин сухо невкусны отдых позитив денег жале хочет приход туд нам рад'

In [182]:
# coding_frequency = CountVectorizer(analyzer='word',
#                                    # binary=True,
#                                    min_df=2,          # минимальное количество вхождений слова
#                                    ngram_range=(2,3),   # какие n-граммы учитывать
#                                    #stop_words=stopwords.words("russian")
#                                   )
#
#res_vectorizer = coding_frequency.fit_transform(tok_stem_text)
#
# таблица частоты слов
#pd.DataFrame(res_vectorizer.toarray(), columns = coding_frequency.vocabulary_.keys())



coding_tfidf = TfidfVectorizer(min_df=2,          # минимальное количество вхождений слова
                                ngram_range=(2,3),   # какие n-граммы учитывать
                                #stop_words=stopwords.words("russian")
                                )

res_vectorizer = coding_tfidf.fit_transform(tok_stem_text)

# таблица частоты слов
pd.DataFrame(res_vectorizer.toarray(), columns = coding_tfidf.vocabulary_.keys())

,вид мор,отношен посетител,оставля жела,жела лучш,люл кебаб,шашлык говядин,хочет приход,нам рад,оставля жела лучш,написа доставк,...,высш уровн,так мо,мыт посуд,запечен картошк,принесл коф,просидел минут,оформлен заказ,воскресен вечер,заведен брал,котлет домашн
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.370297,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [187]:
# Для обучения модели нам нужен список целевых документов
def tagged_document(list_of_ListOfWords):
    for x, ListOfWords in enumerate(list_of_ListOfWords):
        yield doc2vec.TaggedDocument(ListOfWords, [x])

# Обновите модель

# Инициализация модели
d2v_model = doc2vec.Doc2Vec(vector_size=30, # длина вектора, которым будет представлено предложение
                            min_count=2,    # min кол-во встречания слова в прпедложении для учета
                            epochs=30,      # количество эпох
                           )
# новые данные
data_new = list(tagged_document(stem_nltk))
    
# расширить словарный запас
d2v_model.build_vocab(data_new)
  
# Обучение модели Doc2Vec
d2v_model.train(data_new, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)
  
# Анализ выходных данных
# analyze = d2v_model.infer_vector(['Мама мыла раму'])
# analyze

doc2vec_vectorizer = np.array([d2v_model.infer_vector([text_x]) for text_x in tok_stem_text])

In [189]:
scal = MinMaxScaler()
doc2vec_vectorizer = scal.fit_transform(doc2vec_vectorizer)

# Моделирование
- LDA sklearn
- LDA gensim
- LSI sklearn

In [183]:
model = LatentDirichletAllocation(n_components=5,   # количество тем
                                  #learning_method='online',
                                  random_state=42,
                                  n_jobs=-1)

In [190]:
model.fit(doc2vec_vectorizer)   # принимает результат CountVectorizer и аналогичные

LatentDirichletAllocation(n_components=5, n_jobs=-1, random_state=42)

In [185]:
print("Perplexity", model.perplexity(res_vectorizer))
print("Log Likelihood", model.score(res_vectorizer))

Perplexity 7833.7254358155515
Log Likelihood -9009.372145278117


In [191]:
print("Perplexity", model.perplexity(doc2vec_vectorizer))
print("Log Likelihood", model.score(doc2vec_vectorizer))

Perplexity 39.98538330505931
Log Likelihood -27595.01211942672


In [192]:
result = pd.DataFrame(model.transform(doc2vec_vectorizer), columns=[str(i) for i in range(1, 6)])
thems = result.apply(lambda x: x.sort_values().index[-1], axis=1)
result['1'].sort_values(ascending=False)

440    0.958727
270    0.957811
88     0.955991
382    0.955927
438    0.955459
         ...   
268    0.010781
67     0.010685
186    0.010601
143    0.010188
488    0.010108
Name: 1, Length: 500, dtype: float64

In [195]:
df_tmp['text'].iloc[88]

'Позвонили забронировать столик, покурить кальян компанией, девушка администратор сказала, что столик есть и все отлично)\\nМы собрались с ребятами, ехали из Калининграда, путь не ближний, 50 км)\\nВ итоге приехали и ребята на баре развернули нас со словами «все столики заняты и у нас закончился табак» - настроение на вечер было нам обеспечено, уехли из Светлогорска с ужасным настроением. Спасибо, отличный сервис!\\n\n'

In [139]:
def show_topics( vectorizer_x=None, model=None, n_words=20):
    feature_names = np.array(vectorizer_x.get_feature_names_out())
    top_words = []

    for topic_weights in model.components_:
        top_keywords_locs = (-topic_weights).argsort()[:n_words]
        top_words.append(feature_names.take(top_keywords_locs))
    return top_words

In [186]:
pd.DataFrame(show_topics(coding_tfidf, model, 20))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,прост отвратительн,совет заведен,хамск поведен,заказ минут,цен поид,уровен обслуживан,пицц вкусн,прост невозможн,перв очеред,заказ ответ,посуд грязн,фо бо,высок цен,забронирова стол,ден рожден,друг мест,готов вкусн,некотор врем,вечер испорч,половин стол
1,дан заведен,оставля жела лучш,ед вкусн,оставля жела,жела лучш,ден рожден,книг жалоб,написа доставк,намн вкусн,горазд вкусн,следующ ден,отдельн истор,врем ожидан,ожидан заказ,вкусн мяс,вкусн кухн,гост так,минут нам,заказ первы,ситуац прост
2,салат цезар,дан заведен,жив музык,ужасн обслуживан,поид туд,слоен тест,реш посет,отвратительн качеств,лист салат,наш стол,так вообщ,друг гост,ужасн мест,одн мест,мо отз,хуж стал,красив мест,вкус порц,пят минут,заказа ролл
3,ед вкусн,ужасн ед,единственны плюс,сотрудник сожален,долг обслуживан,сыр тест,сказа закрыт,сюд точн,каф каф,прост ужас,одн слов,ролл вкусн,написа сказа,посещен дан заведен,посещен дан,сам ед,ход туд,ве ден,желан приход,дан заведен
4,картошк фри,одн звезд,прост ужас,детск комнат,так ощущен,сдела заказ,ужасн обслуживан,пок сам,куша сам,туд ного,заказ ждал,ед понрав,испорчен настроен,сво заказ,так обслуживан,хамск тон,ужасн мест,знает так,остав отз,заведен хорош


## gensim

In [ ]:
dictionary = corpora.Dictionary(processed_data)
corpus = [dictionary.doc2bow(l) for l in processed_data]

# Метрики

# Сбор всего в Pipeline